In [1]:
import matplotlib.pyplot as plt
import numpy as np
import h5py
import pandas as pd
from scipy import ndimage
from PIL import Image
import ipywidgets as widgets
from ipywidgets import interact, interactive
from pigeon import annotate
from IPython.display import display, Image
import pickle
import sys

sys.path.append('/Users/cgroschner/Documents/HTTEM/segmentation/')

import realDataProcess as rdp

In [2]:
right_images = np.load('20200514_right__Chiral_D_Large_TIFF_Cropped_four_rows_sel_NPs_rotated.npy')

In [3]:
img = [i for i in right_images]

In [4]:
%matplotlib notebook

In [7]:
annotations2 = annotate(
  img,
  options=['good','bad'],
  display_fn=lambda filename: plt.imshow(filename,cmap='gray'))

HTML(value='0 examples annotated, 384 examples left')

Output()

Annotation done.


In [5]:
annotations = annotate(
  img,
  options=['good','bad'],
  display_fn=lambda filename: plt.imshow(filename,cmap='gray'))

HTML(value='0 examples annotated, 384 examples left')

Output()

In [5]:
annotations

[(array([[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]]), 'good'),
 (array([[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]]), 'bad'),
 (array([[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]]), 'good'),
 (array([[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0

In [6]:
right_images.shape

(383, 128, 128)

In [7]:
img = [right_images[idx] for idx in np.arange(0,len(right_images)) if idx%3 == 0]

In [6]:
annotations2 = annotate(
  img,
  options=['good','bad'],
  display_fn=lambda filename: plt.imshow(filename,cmap='gray'))

HTML(value='0 examples annotated, 384 examples left')

Output()

In [12]:
just_labels = []
for i in annotations2:
    just_labels.append(i[1])

In [15]:
just_pics = []
for i in annotations2:
    just_pics.append(i[0])

In [17]:
just_labels_df = pd.DataFrame(data=just_labels)

just_labels_df.iloc[:,0].value_counts()

right_particles_notrot = []
# left_particles_notrot = []
garbage_particles_notrot = []
for idx,lbl in enumerate(just_labels):
    if lbl == 'good':
        right_particles_notrot.append(just_pics[idx])
    elif lbl == 'bad':
        garbage_particles_notrot.append(just_pics[idx])

In [18]:
right_particles_notrot = np.array(right_particles_notrot)


garbage_particles_notrot = np.array(garbage_particles_notrot)

In [19]:
np.save('20200912_right_Chiral_D_Large_TIFF_Cropped_four_rows_sel_NPs_rotated.npy',right_particles_notrot)
# np.save('20200514_left__Chiral_D_Large_TIFF_Cropped_four_rows_sel_NPs_rotated.npy',left_particles_notrot)
np.save('20200912_garbage__Chiral_D_Large_TIFF_Cropped_four_rows_sel_NPs_rotated.npy',garbage_particles_notrot)

In [20]:
right_particles_notrot.shape

(113, 128, 128)

In [35]:
split = int(113*0.4)

In [36]:
split

45

In [37]:
113-45

68

In [46]:
int(113*.2)

22

In [30]:
91*2

182

In [31]:
182*0.2

36.4

In [32]:
91*0.2

18.2

In [43]:
plt.imshow(right_particles_notrot[10])

In [39]:
from sklearn import utils


In [57]:
new_left_images = []
new_left_labels = []
new_right_images = []
new_right_labels = []


for img in right_particles_notrot:
    new_left_images.append(np.fliplr(img))
    new_left_labels.append([0,1])
    new_right_images.append(img)
    new_right_labels.append([1,0])
new_left_images = np.array(new_left_images)
new_right_images = np.array(new_right_images)



right_img_shuff, right_label_shuff = utils.shuffle(new_right_images[:68], new_right_labels[:68],random_state=0)
left_img_shuff, left_label_shuff = utils.shuffle(new_left_images[:68], new_left_labels[:68],random_state=0)

X_train = np.concatenate((right_img_shuff[:68],left_img_shuff[:68]),axis =0)
Y_train = np.concatenate((right_label_shuff[:68],left_label_shuff[:68]),axis = 0)



right_img_shuff, right_label_shuff = utils.shuffle(new_right_images[68:91], new_right_labels[68:91],random_state=0)
left_img_shuff, left_label_shuff = utils.shuffle(new_left_images[68:91], new_left_labels[68:91],random_state=0)


X_test = np.concatenate((right_img_shuff,left_img_shuff),axis = 0)
Y_test = np.concatenate((right_label_shuff,left_label_shuff),axis = 0)

X_train_norm = X_train/X_train.max()
X_test_norm = X_test/X_test.max()
X_train_norm = np.expand_dims(X_train_norm,axis=3)
X_test_norm = np.expand_dims(X_test_norm,axis=3)

X_train_shuff, Y_train_shuff = utils.shuffle(X_train_norm, Y_train,random_state=0)
X_test_shuff, Y_test_shuff = utils.shuffle(X_test_norm, Y_test,random_state=0)

X_val = np.concatenate((new_right_images[91:],new_left_images[91:]),axis =0)
Y_val = np.concatenate((new_right_labels[91:],new_left_labels[91:]),axis=0)


i = h5py.File('20200912_0perror_Chiral_D_Large_TIFF_Cropped_four_rows_sel_NPs_rotated.h5','w')
i.create_dataset('trainX',data=X_train_shuff)
i.create_dataset('trainY',data=Y_train_shuff)
i.create_dataset('testX',data=X_test_shuff)
i.create_dataset('testY',data=Y_test_shuff)
i.create_dataset('valX',data=X_val)
i.create_dataset('valY',data=Y_val)
i.close()

In [52]:
113 - 22

91

In [53]:
91-68

23

In [54]:
23*0.05

1.1500000000000001

In [51]:
68+22

90

In [58]:
new_left_images = []
new_left_labels = []
new_right_images = []
new_right_labels = []


for img in right_particles_notrot:
    new_left_images.append(np.fliplr(img))
    new_left_labels.append([0,1])
    new_right_images.append(img)
    new_right_labels.append([1,0])
new_left_images = np.array(new_left_images)
new_right_images = np.array(new_right_images)

split = int(68*0.05)
for idx in np.arange(0,split):
    new_right_images[idx] = np.fliplr(new_right_images[idx])

for idx in np.arange(0,split):
    new_left_images[idx] = np.fliplr(new_left_images[idx])

right_img_shuff, right_label_shuff = utils.shuffle(new_right_images[:68], new_right_labels[:68],random_state=0)
left_img_shuff, left_label_shuff = utils.shuffle(new_left_images[:68], new_left_labels[:68],random_state=0)

X_train = np.concatenate((right_img_shuff[:68],left_img_shuff[:68]),axis =0)
Y_train = np.concatenate((right_label_shuff[:68],left_label_shuff[:68]),axis = 0)

split = int(23*0.05)
for idx in np.arange(68,split+68):
    new_right_images[idx] = np.fliplr(new_right_images[idx])

for idx in np.arange(68,split+68):
    new_left_images[idx] = np.fliplr(new_left_images[idx])

right_img_shuff, right_label_shuff = utils.shuffle(new_right_images[68:91], new_right_labels[68:91],random_state=0)
left_img_shuff, left_label_shuff = utils.shuffle(new_left_images[68:91], new_left_labels[68:91],random_state=0)


X_test = np.concatenate((right_img_shuff,left_img_shuff),axis = 0)
Y_test = np.concatenate((right_label_shuff,left_label_shuff),axis = 0)

X_train_norm = X_train/X_train.max()
X_test_norm = X_test/X_test.max()
X_train_norm = np.expand_dims(X_train_norm,axis=3)
X_test_norm = np.expand_dims(X_test_norm,axis=3)

X_train_shuff, Y_train_shuff = utils.shuffle(X_train_norm, Y_train,random_state=0)
X_test_shuff, Y_test_shuff = utils.shuffle(X_test_norm, Y_test,random_state=0)

X_val = np.concatenate((new_right_images[91:],new_left_images[91:]),axis =0)
Y_val = np.concatenate((new_right_labels[91:],new_left_labels[91:]),axis=0)


i = h5py.File('20200912_5perror_Chiral_D_Large_TIFF_Cropped_four_rows_sel_NPs_rotated.h5','w')
i.create_dataset('trainX',data=X_train_shuff)
i.create_dataset('trainY',data=Y_train_shuff)
i.create_dataset('testX',data=X_test_shuff)
i.create_dataset('testY',data=Y_test_shuff)
i.create_dataset('valX',data=X_val)
i.create_dataset('valY',data=Y_val)
i.close()





In [59]:
new_left_images = []
new_left_labels = []
new_right_images = []
new_right_labels = []


for img in right_particles_notrot:
    new_left_images.append(np.fliplr(img))
    new_left_labels.append([0,1])
    new_right_images.append(img)
    new_right_labels.append([1,0])
new_left_images = np.array(new_left_images)
new_right_images = np.array(new_right_images)

split = int(68*0.1)
for idx in np.arange(0,split):
    new_right_images[idx] = np.fliplr(new_right_images[idx])

for idx in np.arange(0,split):
    new_left_images[idx] = np.fliplr(new_left_images[idx])

right_img_shuff, right_label_shuff = utils.shuffle(new_right_images[:68], new_right_labels[:68],random_state=0)
left_img_shuff, left_label_shuff = utils.shuffle(new_left_images[:68], new_left_labels[:68],random_state=0)

X_train = np.concatenate((right_img_shuff[:68],left_img_shuff[:68]),axis =0)
Y_train = np.concatenate((right_label_shuff[:68],left_label_shuff[:68]),axis = 0)

split = int(23*0.1)
for idx in np.arange(68,split+68):
    new_right_images[idx] = np.fliplr(new_right_images[idx])

for idx in np.arange(68,split+68):
    new_left_images[idx] = np.fliplr(new_left_images[idx])

right_img_shuff, right_label_shuff = utils.shuffle(new_right_images[68:91], new_right_labels[68:91],random_state=0)
left_img_shuff, left_label_shuff = utils.shuffle(new_left_images[68:91], new_left_labels[68:91],random_state=0)


X_test = np.concatenate((right_img_shuff,left_img_shuff),axis = 0)
Y_test = np.concatenate((right_label_shuff,left_label_shuff),axis = 0)

X_train_norm = X_train/X_train.max()
X_test_norm = X_test/X_test.max()
X_train_norm = np.expand_dims(X_train_norm,axis=3)
X_test_norm = np.expand_dims(X_test_norm,axis=3)

X_train_shuff, Y_train_shuff = utils.shuffle(X_train_norm, Y_train,random_state=0)
X_test_shuff, Y_test_shuff = utils.shuffle(X_test_norm, Y_test,random_state=0)

X_val = np.concatenate((new_right_images[91:],new_left_images[91:]),axis =0)
Y_val = np.concatenate((new_right_labels[91:],new_left_labels[91:]),axis=0)


i = h5py.File('20200912_10perror_Chiral_D_Large_TIFF_Cropped_four_rows_sel_NPs_rotated.h5','w')
i.create_dataset('trainX',data=X_train_shuff)
i.create_dataset('trainY',data=Y_train_shuff)
i.create_dataset('testX',data=X_test_shuff)
i.create_dataset('testY',data=Y_test_shuff)
i.create_dataset('valX',data=X_val)
i.create_dataset('valY',data=Y_val)
i.close()





In [60]:
new_left_images = []
new_left_labels = []
new_right_images = []
new_right_labels = []


for img in right_particles_notrot:
    new_left_images.append(np.fliplr(img))
    new_left_labels.append([0,1])
    new_right_images.append(img)
    new_right_labels.append([1,0])
new_left_images = np.array(new_left_images)
new_right_images = np.array(new_right_images)

split = int(68*0.2)
for idx in np.arange(0,split):
    new_right_images[idx] = np.fliplr(new_right_images[idx])

for idx in np.arange(0,split):
    new_left_images[idx] = np.fliplr(new_left_images[idx])

right_img_shuff, right_label_shuff = utils.shuffle(new_right_images[:68], new_right_labels[:68],random_state=0)
left_img_shuff, left_label_shuff = utils.shuffle(new_left_images[:68], new_left_labels[:68],random_state=0)

X_train = np.concatenate((right_img_shuff[:68],left_img_shuff[:68]),axis =0)
Y_train = np.concatenate((right_label_shuff[:68],left_label_shuff[:68]),axis = 0)

split = int(23*0.2)
for idx in np.arange(68,split+68):
    new_right_images[idx] = np.fliplr(new_right_images[idx])

for idx in np.arange(68,split+68):
    new_left_images[idx] = np.fliplr(new_left_images[idx])

right_img_shuff, right_label_shuff = utils.shuffle(new_right_images[68:91], new_right_labels[68:91],random_state=0)
left_img_shuff, left_label_shuff = utils.shuffle(new_left_images[68:91], new_left_labels[68:91],random_state=0)


X_test = np.concatenate((right_img_shuff,left_img_shuff),axis = 0)
Y_test = np.concatenate((right_label_shuff,left_label_shuff),axis = 0)

X_train_norm = X_train/X_train.max()
X_test_norm = X_test/X_test.max()
X_train_norm = np.expand_dims(X_train_norm,axis=3)
X_test_norm = np.expand_dims(X_test_norm,axis=3)

X_train_shuff, Y_train_shuff = utils.shuffle(X_train_norm, Y_train,random_state=0)
X_test_shuff, Y_test_shuff = utils.shuffle(X_test_norm, Y_test,random_state=0)

X_val = np.concatenate((new_right_images[91:],new_left_images[91:]),axis =0)
Y_val = np.concatenate((new_right_labels[91:],new_left_labels[91:]),axis=0)


i = h5py.File('20200912_20perror_Chiral_D_Large_TIFF_Cropped_four_rows_sel_NPs_rotated.h5','w')
i.create_dataset('trainX',data=X_train_shuff)
i.create_dataset('trainY',data=Y_train_shuff)
i.create_dataset('testX',data=X_test_shuff)
i.create_dataset('testY',data=Y_test_shuff)
i.create_dataset('valX',data=X_val)
i.create_dataset('valY',data=Y_val)
i.close()



